## ***爬取京东商品评价
https://blog.csdn.net/cabigge_bear/article/details/100119106

### Python爬虫：爬取京东商品评论(处理json) urllib3+bs4+sqlite3
https://blog.csdn.net/qq_39192827/article/details/86747053

### **ModuleNotFoundError: No module named 'fake_useragent'
- https://blog.csdn.net/weixin_42201745/article/details/80350831

打开cmd命令行，不用进入python（我开始直接进入了python，所以也没有成功安装），直接在上面写入就可以安装上fake-useragent库了：
> pip install fake-useragent

In [1]:
#!/usr/bin/env python
# -*- coding:utf-8 -*-
import re, requests, csv, codecs, json
from requests.exceptions import RequestException
from bs4 import BeautifulSoup
from urllib.parse import urlencode

In [2]:
# code for test
url = 'https://sclub.jd.com/comment/productPageComments.action?productId=100005171461&score=0&sortType=5&page=0&pageSize=10&isShadowSku=0&fold=1'
headers = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36',
    'referer':'https://item.jd.com/100005171461.html'
}
res = requests.get(url, headers=headers)
print(res.status_code)  # 返回的状态码是200
#print(res.text) 

200


In [3]:
def get_html(pID, page):  # 返回评论真实的网页内容
    url = f"https://sclub.jd.com/comment/productPageComments.action?productId={pID}&score=0&sortType=5&page={page}&pageSize=10&isShadowSku=0&fold=1"
    headers = {
        'user-agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36',
        f'referer':'https://item.jd.com/{pID}.html'
    }
    res = requests.get(url, headers=headers)
    return res.text

In [4]:
# code for test
data = json.loads(get_html('100005171461','0'))
#print(data)

for i in data['comments']:
    print(i['content'])
    print('----- ----- -----')

京东物流真是太给力了，0点下单的，下午就到了，也太快了，包装很好，很喜欢，颜值很高，电脑很好！
首先，这台电脑很轻很薄，很轻巧，可谓是小巧玲珑，非常方便外出携带办公使用！之前自己的电脑非常大，带出去是非常不方便的，这个新买的电脑很小，很好！非常方便携带了！
其次，这台电脑的内存是12G+512G的，这个内存是非常够使用的，打游戏，办公都非常合适使用，但是不保证根据自己的需要下载其他自己所需要的应用的，不过应该是够使用的。
最后，我发现我之前使用的电脑非常容易发热，这个新买的电脑就不会发热，所以这点是非常喜欢的，很好，非常适合大家使用！
综上所述，这个电脑还可以，适合使用！
----- ----- -----
轻薄程度：用了八佰的活动圏。叟嗦拱 肿号：{沐**沐.省} 领取的。颜色超好看，能对电脑起到很好的保护作用，打字的时候键盘会blingbling(?????)超好看！物流很快，1天就到了。包装也很好，用好多东西包着呢，完全不用担心电脑会受损。颜色很适合美美哒小仙女们，推荐购买！首先，这台电脑很轻很薄很小巧，蛮适合携带的！ 其次，这台电脑的内存是12G+512G的！这个内存是够用的！但是不能完全保证不下载其他的呢！ 除此之外，这台电脑，emm，这家店的所有款式的电脑的硬盘是没有分盘的，只有一个系统C盘！所以只能自己去分盘！而且如果你下载了东西到C盘，是不可以随便删除的！会导致系统崩溃！然后东西，内存占的大了电脑也会卡顿！ 最后，我看到很多人说电脑发热！其实现在看来还好！不容易发热！主要看用的什么软件！如果打游戏的话肯定是很容易发热的！而这款说的是游戏本吧！但是我觉得这款电脑不适合打游戏！内寸不行！散热性或许也不好吧！ 所以，综上所述！这款电脑还行，只是电脑内存我觉得真的不太够！！笔记本性能不错 很流畅 外观设计很漂亮 非常的轻薄体积小方便携带 开机时间短 指纹解锁设计很不错? 大内存 性
----- ----- -----
轻薄程度：用了八佰的活动***。叟嗦拱 肿号：{***} 领取的。颜色超好看，能对电脑起到很好的保护作用，打字的时候键盘会blingbling(?????)超好看！物流很快，1天就到了。包装也很好，用好多东西包着呢，完全不用担心电脑会受损。颜色很适合美美哒小仙女们，推荐购买！
首先，这台电脑很轻很薄很小巧，蛮适合携带的！ 其次，这台电脑的内存是

In [13]:
def get_comment(product_ID):
    for p in range(10):
        data = json.loads(get_html(product_ID,p)) #'100005171461'
        #print(data)
        for d in data['comments']:
            print(d['content'])
            print('----- ----- -----')

In [6]:
# 1、要取所有的电脑数据，首先得获得所有电脑的网页源码，以及url地址如下：
def download(url, headers, num_retries=3):
    print("download", url)
    try:
        response = requests.get(url, headers=headers)
        print(response.status_code)
        # 如果相应的状态码为：200 那么return 返回网站源码，如果访问非200则调用重试
        if response.status_code == 200:
            return response.content
        return None
    except RequestException as e:
        print(e.response)
        html = ""
        if hasattr(e.response, 'status_code'):
            code = e.response.status_code
            print('error code', code)
            if num_retries > 0 and 500 <= code < 600:
                html = download(url, headers, num_retries - 1)
        else:
            code = None
    return html

In [7]:
def get_json(url):
    data = requests.get(url).json()
    #print(data)
    result = data['CommentsCount']
    for i in result:
        return i["CommentCountStr"], i["GoodRateShow"]

In [8]:
def write_csv(csv_name, data_list):
    with open(csv_name, 'w', newline='', encoding='utf-8-sig') as f:
        writer = csv.writer(f)
        fields = ('ID', 'NAME', 'PRICE(元)', 'Comment#', 'Good Ratio')
        writer.writerow(fields)
        for data in data_list:
            writer.writerow(data)

In [9]:
def find_Computer(url, headers):
    r = download(url, headers=headers)
    #print(r)
    page = BeautifulSoup(r, "lxml")
    #print(page.prettify())
   
    all_items = page.find_all('li', attrs={'class':'gl-item'})
    data_list = []
    
    for all in all_items:
        Computer_id = all["data-sku"]
        Computer_name = all.find('div', attrs={'class': 'p-name p-name-type-2'}).find('em').text
        _price = all.find('div', attrs={'class': 'p-price'}).find('strong')
        Computer_price = _price.get('data-price', _price.find('i').text)

        # 取每台電腦的Json數據(包含 評價等等信息)
        Comment = f"https://club.jd.com/comment/productCommentSummaries.action?referenceIds={Computer_id}"
        comment_count, good_rate = get_json(Comment)

        #print(f"ID：{Computer_id}")
        #print(f"Name：{Computer_name}")
        #print(f"Price：{Computer_price}元")
        #print(f"Comment#：", comment_count)
        #print(f"Good Ratio：{good_rate}\n")
        row = []
        row.append(Computer_id)
        row.append(Computer_name)
        row.append(str(Computer_price))
        row.append(comment_count)
        row.append(good_rate)
        data_list.append(row)
    return data_list

In [10]:
def main(page_end):
    headers = {
    'User-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.87 Safari/537.36",
    "referer": "https://search.jd.com"
    }
    all_list = []
    
    for page in range(1,page_end*2+1,2):
        URL = f"https://search.jd.com/Search?keyword=lenovo&enc=utf-8&qrst=1&rt=1&stop=1&vt=2&bs=1&wq=lenovo&ev=exbrand_%E8%81%94%E6%83%B3%EF%BC%88Lenovo%EF%BC%89%5E&page={page}&s=1&click=0"
        data_list = find_Computer(URL, headers=headers)
        all_list += data_list
        write_csv("leovo.csv", all_list)

In [11]:
if __name__ == '__main__':
    main(10)
    print("\nDONE\n")

download https://search.jd.com/Search?keyword=lenovo&enc=utf-8&qrst=1&rt=1&stop=1&vt=2&bs=1&wq=lenovo&ev=exbrand_%E8%81%94%E6%83%B3%EF%BC%88Lenovo%EF%BC%89%5E&page=1&s=1&click=0
200
download https://search.jd.com/Search?keyword=lenovo&enc=utf-8&qrst=1&rt=1&stop=1&vt=2&bs=1&wq=lenovo&ev=exbrand_%E8%81%94%E6%83%B3%EF%BC%88Lenovo%EF%BC%89%5E&page=3&s=1&click=0
200
download https://search.jd.com/Search?keyword=lenovo&enc=utf-8&qrst=1&rt=1&stop=1&vt=2&bs=1&wq=lenovo&ev=exbrand_%E8%81%94%E6%83%B3%EF%BC%88Lenovo%EF%BC%89%5E&page=5&s=1&click=0
200
download https://search.jd.com/Search?keyword=lenovo&enc=utf-8&qrst=1&rt=1&stop=1&vt=2&bs=1&wq=lenovo&ev=exbrand_%E8%81%94%E6%83%B3%EF%BC%88Lenovo%EF%BC%89%5E&page=7&s=1&click=0
200
download https://search.jd.com/Search?keyword=lenovo&enc=utf-8&qrst=1&rt=1&stop=1&vt=2&bs=1&wq=lenovo&ev=exbrand_%E8%81%94%E6%83%B3%EF%BC%88Lenovo%EF%BC%89%5E&page=9&s=1&click=0
200
download https://search.jd.com/Search?keyword=lenovo&enc=utf-8&qrst=1&rt=1&stop=1&vt=2&bs=